In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/amrzr/Work/Codes/Offline_IMOEA_Framework/')
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from desdeo_problem.testproblems.DBMOPP.DBMOPP_generator import DBMOPP_generator
from scipy.stats import qmc

import copy
import pickle
import os

In [3]:
data_folder = '/home/amrzr/Work/Codes/data'
init_folder = data_folder + '/constraint_handling_dataset'
problem_folder = init_folder + '/DBMOPP/DBMOPP_problems'
file_instances = init_folder + '/test_instances_DBMOPP.csv'
data_instances = pd.read_csv(file_instances)
problem_testbench = 'DBMOPP'
path = data_folder+'/constraint_handling_dataset/' + problem_testbench 
all_problems = data_instances["problem"].values
all_n_vars = data_instances["nvars"].values
all_objs = data_instances["K"].values
all_sample_size = data_instances["N"].values
all_n_global_pareto_regions = data_instances["n_global_pareto_regions"].values
all_constraint_type = data_instances["constraint_type"].values

n_local_pareto_regions = 0 
n_dominance_res_regions = 0
const_space = 0.0
pareto_set_type = 0 

ndo = 0
vary_sol_density = False
vary_objective_scales = False
prop_neutral = 0
nm = 10000

#all_ndim = data_instances["ndim"].values
#all_bound_valL = data_instances["bound_valL"].values
#all_bound_valU = data_instances["bound_valU"].values
size_instance = np.size(all_problems)
boundoldL= -1 
boundoldU = 1
nruns = 35

In [16]:
for instance in range(size_instance):
    prob = all_problems[instance]
    n_vars = all_n_vars[instance]
    obj = all_objs[instance]
    sample_size = all_sample_size[instance]
    n_global_pareto_regions = all_n_global_pareto_regions[instance]
    constraint_type = all_constraint_type[instance]
    #ndim = all_ndim[instance]
    #bound_valL = all_bound_valL[instance]
    #bound_valU = all_bound_valU[instance]

    problem_spec = prob +'_'+ str(sample_size) + '_' + str(obj) + '_' + \
        str(n_vars) +  '_b'+str(n_global_pareto_regions) +'_' + str(constraint_type)

    #boundvecL = bound_valL* np.ones(n_vars)
    #boundvecU = bound_valU* np.ones(n_vars)
    #boundvecL = boundoldL* np.ones(n_vars)
    #boundvecU = boundoldU* np.ones(n_vars)
    #boundvecL[:ndim]=np.ones(ndim)*bound_valL
    #boundvecU[:ndim]=np.ones(ndim)*bound_valU
    #boundL = np.tile(boundvecL,(sample_size,1))
    #boundU = np.tile(boundvecU,(sample_size,1))

    simple_problem = DBMOPP_generator(
                                    obj,
                                    n_vars,
                                    n_local_pareto_regions,
                                    n_dominance_res_regions,
                                    n_global_pareto_regions,
                                    const_space,
                                    pareto_set_type,
                                    constraint_type, 
                                    ndo, 
                                    vary_sol_density, 
                                    vary_objective_scales, 
                                    prop_neutral, 
                                    nm
                                ) 
    testproblem = simple_problem.generate_problem()
    for run in range(nruns):
        sampler = qmc.LatinHypercube(d=n_vars)
        data = sampler.random(n=sample_size)*(boundoldU-(boundoldL)) + (boundoldL)
        # print(data)

        #testproblem = test_problem_builder(prob, n_of_objectives= obj, n_of_variables=n_vars)
        obj_val = testproblem.evaluate(data)
        
        #failed_loc = np.where(np.all(data >= boundL, axis=1) & np.all(data <= boundU, axis=1))
        
        failed_loc = np.where(np.any(obj_val[2]<=0, axis=1))
        stat_success = np.ones((sample_size,1))
        stat_success[failed_loc,0]=0
        data_success = data[np.where(stat_success==1)[0],:]

        data_failed=data[failed_loc[0],:]
        obj_vals = obj_val[0]
        obj_success = obj_vals[np.where(stat_success==1)[0],:]


        #stat_success = np.ones((sample_size,1))
        #stat_success[failed_loc,0]=0

        #data_failed=data[failed_loc[0],:]
        #ax.scatter(data_failed[:,0],data_failed[:,1],data_failed[:,2], c='black', marker=',')
        #obj_vals = obj_val[0]
        #obj_success = obj_vals[np.where(stat_success==1)[0],:]

        #np.size(failed_loc)
        #print(np.size(failed_loc))

        data_class = pd.DataFrame(np.hstack((data, stat_success)))
        data_success = pd.DataFrame(np.hstack((data[np.where(stat_success==1)[0],:],obj_success)))
        data_all = pd.DataFrame(np.hstack((np.hstack((data, obj_vals)),stat_success)))

        data_class.to_csv(path + '/' + problem_spec + '_'+ str(run)+ '_data_class.csv',index=False)
        data_success.to_csv(path + '/' + problem_spec + '_'+ str(run) + '_data_success.csv',index=False)
        data_all.to_csv(path + '/' + problem_spec  + '_'+ str(run) + '_data_all.csv',index=False)

    #testproblem_copy = copy.deepcopy(testproblem)
    simple_problem_copy=copy.deepcopy(simple_problem)
        #classification_model_copy = copy.deepcopy(classification_model)
    models_dict = {#'problem': testproblem,
                    'simple_problem' : simple_problem_copy}
    model_file=problem_spec
    outfile = open(problem_folder+'/' + model_file, 'wb')
    pickle.dump(models_dict, outfile, protocol=3)
    outfile.close()

Assigning any checker soft/hard constraint regions and neutral regions

Assigning any vertex soft/hard constraint regions

Assigning any centre soft/hard constraint regions.

Assigning any moat soft/hard constraint regions

Generating MOProblem
Assigning any checker soft/hard constraint regions and neutral regions

Assigning any vertex soft/hard constraint regions

Assigning any centre soft/hard constraint regions.

Assigning any moat soft/hard constraint regions

Generating MOProblem
Assigning any checker soft/hard constraint regions and neutral regions

Assigning any vertex soft/hard constraint regions

Assigning any centre soft/hard constraint regions.

Assigning any moat soft/hard constraint regions

Generating MOProblem
Assigning any checker soft/hard constraint regions and neutral regions

Assigning any vertex soft/hard constraint regions

Assigning any centre soft/hard constraint regions.

Assigning any moat soft/hard constraint regions

Generating MOProblem
Assigning any checke